In [1]:

%matplotlib qt
import mne
import numpy as np
import matplotlib
# Establecer un backend interactivo, como 'Qt5Agg', 'GTK3Agg', etc.
# Esto depende de los backends disponibles en tu sistema.

import matplotlib.pyplot as plt

matplotlib.use('Qt5Agg')  # Asegúrate de que este backend está instalado.
mne.viz.set_browser_backend('qt')  # Activa el backend interactivo

import pandas as pd 
import os
import sys

from mne.preprocessing import ICA, corrmap, create_ecg_epochs, create_eog_epochs

from os.path import join as pathjoin
from time import time

from pathlib import Path

from autoreject import AutoReject

from neurodsp.spectral import compute_spectrum, trim_spectrum
from neurodsp.plts import plot_power_spectra

# Import IRASA related functions
from neurodsp.aperiodic import compute_irasa, fit_irasa

Using qt as 2D backend.


In [2]:
# Variables path
layer_script = "block"

#process=""
disco="g"

subj = "S01"

#subj = sys.argv[1] ## name of participant list

# Carpeta general
datadir = Path(f"{disco}:\\PROYECTO_SELF")

# #carpetas generales de datos
brain_vision_dir = datadir /"Data_self"/"Export_acw_Brain_Vision"
fif_data = datadir /"Data_self"/ "fif_data" 


#carpeta analysis
output_analysis= datadir / "output_analysis"

acw_analysis_path = output_analysis / "acw"
acw_analysis_path.mkdir(parents=True, exist_ok=True)

# # Carpeta de preprocesado
# output_preproc = datadir / "output_preproc"

# preproc_path = output_preproc / f"preproc_{layer_script}"
# preproc_path.mkdir(parents=True, exist_ok=True)
 
# # Carpeta de epocas "sucias"
# epochs_path = preproc_path / f"epochs_{layer_script}"
# epochs_path.mkdir(parents=True, exist_ok=True) 

# # Carpeta de ICA
# ICA_path = preproc_path / f"ICA_{layer_script}"
# ICA_path.mkdir(parents=True, exist_ok=True)

# # Épocas limpias
# epochs_clean_path = preproc_path / f"epochs_clean_{layer_script}"
# epochs_clean_path.mkdir(parents=True, exist_ok=True)

# #epocas epochs
# epochs_path = Path(preproc_path) / f"epochs_{layer_script}"
# epochs_path.mkdir(parents=True, exist_ok=True)
 

# # Definir la carpeta de output_source antes de usarla
# output_source = Path(r"g:\MOUS_204\output_source")

# source_path = output_source / f"source_{layer_script}"
# source_path.mkdir(parents=True, exist_ok=True)

# #raw_hsp es el raw con fiducials cargados
# raw_hsp_path = source_path / f"raw_hsp"
# raw_hsp_path.mkdir(parents=True, exist_ok=True)

# # Carpeta de forward solution
# fwd_path = source_path / f"fwd"
# fwd_path.mkdir(parents=True, exist_ok=True)

# # Carpeta de inverse solution
# inverse_path = source_path / f"inverse"
# inverse_path.mkdir(parents=True, exist_ok=True)


# mne.utils.set_config('SUBJECTS_DIR', r'\\wsl$\Ubuntu-20.04\usr\local\freesurfer\subjects', set_env=True)


In [3]:

#epochs
combinaciones = ["S1 - S4","S1 - S5", "S2 - S4", "S2 - S5", "S3 - S4", "S3 - S5"]

combinacion= combinaciones[1]

# Obtener todos los archivos y carpetas dentro de la ruta
subjects = list(fif_data.iterdir())

# Filtrar solo los nombres de sujetos (removiendo "_epochs")
subject_names = [subject.stem.replace("_epochs", "") for subject in subjects]

subj=subject_names[0]
subj


# # epochs_zinnen_path = epochs_clean_path / f"{subj}_epochs_zinnen_{type_script}"
# # epochs_path = epochs_clean_path / f"{subj}_epochs_{type_script}"

path_epochs= fif_data / f"{subj}_epochs" / f"{subj}_{combinacion}-epo.fif"

epochs = mne.read_epochs(path_epochs) 


Reading g:\PROYECTO_SELF\Data_self\fif_data\S01_epochs\S01_S1 - S5-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9198.00 ms
        0 CTF compensation matrices available
Not setting metadata
21 matching events found
No baseline correction applied
0 projection items activated


In [ ]:
## obtener power spechtum de el objeto evoked -- esto serñía el MIXED POWER 

# psds=evoked.compute_psd(method='welch', fmin=fmin, fmax=fmax, reject_by_annotation=True, n_fft = int(6 * sfreq))
# psds_all.append(psds)
            
# psds,freqs= mne.time_frequency.psd_array_welch(array, sfreq, fmin=0, fmax=inf, n_fft=256, n_overlap=0, 
# #                                    n_per_seg=None, n_jobs=None, average='mean', window='hamming', remove_dc=True, *, 
# #                                    output='power', verbose=None)



##please note THAT THIS IS ONLY FOR EVOKED, YOU NEED TO CALCULATE IT THROUGH EPOCHS

# ##def compute_psd(self, fmin=0, fmax=np.inf, tmin=None, tmax=None, proj=False)
# array=evoked.get_data()
# sfreq=evoked.info['sfreq']
# fmin=0.5
# fmax=40
# nfft=1024 # to increase the spectral resolution, , although you can go to 4096 if you want to see more details
# njobs=10

# ##compute psd FOR WHOLE SIGNAL
# psds,freqs= mne.time_frequency.psd_array_welch(array, sfreq, fmin=fmin, fmax=fmax, n_fft=nfft, n_jobs=njobs)



NameError: name 'evoked' is not defined

In [12]:
def compute_ple(epochs, f_range=(0.1, 40), hset=None, thresh=None, isplot=False):
    # Filtrado de datos
    # Importa el objeto epochs y obtiene los datos SOLO DE EEG
    data_epochs = epochs.copy().pick("eeg").get_data()

    #get duration
    duration= epochs.tmax - epochs.tmin
    #get sample frequency
    sfreq= epochs.info['sfreq']

    # ##valores de todos los electrodos en una epoca
    # freqs_elect_all_epoch=[]
    # psd_aperiodic_elect_all_epoch=[]
    # psd_periodic_elect_all_epoch=[]

    # intercept_y_elect_all_epoch=[]
    # slope_elect_all_epoch=[]
    psd_periodict_elect_all_epoch_all=[]
    psd_aperiodic_elect_all_epoch_all=[]
    slope_elect_all_epoch_all=[]
    intercept_y_elect_all_epoch_all=[]

    #for each epoch
    for i in range(0,len(data_epochs)):
        data_epoch=data_epochs[i]
        #print(f"una epoca tiene un shape de {data_epoch.shape}, con {data_epoch.shape[0]} canales y {data_epoch.shape[1]} puntos de tiempo")




        ##valores de ALL ELECT en ONE EPOCH
        psd_aperiodic_elect_all_epoch=[]
        psd_periodic_elect_all_epoch=[]

        intercept_y_elect_all_epoch=[]
        slope_elect_all_epoch=[]



        #for each sensor on each epoch
        for i in range(0,len(data_epoch)):
            #en IRASA hay que meter el time series, es decir, el time series de un canal    
            data_elect=data_epoch[i]
            #print(f"un canal tiene un shape de {data_elect.shape} con {data_elect.shape[0]} puntos de tiempo")


            ##computar IRASA para un canal, obtener componentes aperiodicas (FRACTAL) y periodicas (OSCILATORIO)

            # freqs,psd_aperiodic, psd_periodic=neurodsp.aperiodic.irasa.compute_irasa(sig, fs, f_range=None,
            # hset=None, thresh=None, **spectrum_kwargs)
            
                    # f_range # Frequency range to restrict the analysis to. tuple, optional
                    # hset # Resampling factors used in IRASA calculation.
                    #  If not provided, defaults to values from 1.1 to 1.9 with an increment of 0.05.  1d array

            #remember to set this value for function, in order to be able to modify it at demmand

            freqs,psd_aperiodic_elect_epoch, psd_periodic_elect_epoch=compute_irasa(data_elect, fs=sfreq, f_range=f_range,  
                                                                                    hset=hset, thresh=thresh)

            ##fit IRASA para un canal
            #intercept, slope=fit_irasa(freqs, psd_aperiodic)
            #intercept es el valor de la pendiente en el eje y, es un indicador de la ampplitud
            #slope es la pendiente, es el POWER LAW EXPONENT, 


            intercept_y_elect_epoch, slope_elect_epoch = fit_irasa(freqs, psd_aperiodic_elect_epoch)

            ##append values for each electrode in ONE epoch
            psd_aperiodic_elect_all_epoch.append(psd_aperiodic_elect_epoch)
            psd_periodic_elect_all_epoch.append(psd_periodic_elect_epoch)

            intercept_y_elect_all_epoch.append(intercept_y_elect_epoch)
            slope_elect_all_epoch.append(slope_elect_epoch)

        

        ##append values for each electrode in ALL epochs
        psd_periodict_elect_all_epoch_all.append(psd_periodic_elect_all_epoch)
        psd_aperiodic_elect_all_epoch_all.append(psd_aperiodic_elect_all_epoch)
        
        intercept_y_elect_all_epoch_all.append(intercept_y_elect_all_epoch)
        slope_elect_all_epoch_all.append(slope_elect_all_epoch)



    ##promedio de todos los valores de los electrodos en todas las epocas
        ##conversiñón en arrays
    psd_periodict_elect_all_epoch_all_array=np.array(psd_periodict_elect_all_epoch_all)
    psd_aperiodic_elect_all_epoch_all_array=np.array(psd_aperiodic_elect_all_epoch_all)

    intercept_y_elect_all_epoch_all_array=np.array(intercept_y_elect_all_epoch_all)
    slope_elect_all_epoch_all_array=np.array(slope_elect_all_epoch_all)

    ##computo de la media de TODOS los electrodos en CADA EPOCA
    psd_periodict_elect_mean_epoch_all=np.mean(psd_periodict_elect_all_epoch_all_array, axis=1)
    psd_aperiodic_elect_mean_epoch_all=np.mean(psd_aperiodic_elect_all_epoch_all_array, axis=1)

    intercept_y_elect_mean_epoch_all=np.mean(intercept_y_elect_all_epoch_all_array, axis=1)
    slope_elect_mean_epoch_all=np.mean(slope_elect_all_epoch_all_array, axis=1)


    ##computo de la media de CADA los electrodos en TODAS EPOCAS
    psd_periodict_elect_all_epoch_mean=np.mean(psd_periodict_elect_all_epoch_all_array, axis=0)
    psd_aperiodic_elect_all_epoch_mean=np.mean(psd_aperiodic_elect_all_epoch_all_array, axis=0)

    intercept_y_elect_all_epoch_mean=np.mean(intercept_y_elect_all_epoch_all_array, axis=0)
    slope_elect_all_epoch_mean=np.mean(slope_elect_all_epoch_all_array, axis=0)




    table_PLE = pd.DataFrame({
        'Subject': [subj] * len(psd_periodict_elect_all_epoch_all),  # Repite el sujeto para todas las filas
        'Combination': [combinacion] * len(psd_periodict_elect_all_epoch_all),  # Repite la combinación para todas las filas
        "Epoch": range(1, len(psd_periodict_elect_all_epoch_all) + 1),  # Número de épocas
        "freqs":[freqs]*len(psd_periodict_elect_all_epoch_all),
        'psd_periodict_elect_all_epoch_all': psd_periodict_elect_all_epoch_all,
        "psd_aperiodic_elect_all_epoch_all": psd_aperiodic_elect_all_epoch_all,
        'intercept_y_elect_all_epoch_all': intercept_y_elect_all_epoch_all,
        'slope_elect_all_epoch_all': slope_elect_all_epoch_all,
        'psd_periodict_elect_mean_epoch_all': [psd_periodict_elect_mean_epoch_all] * len(psd_periodict_elect_all_epoch_all),
        'psd_aperiodic_elect_mean_epoch_all': [psd_aperiodic_elect_mean_epoch_all] * len(psd_periodict_elect_all_epoch_all),
        "intercept_y_elect_mean_epoch_all": [intercept_y_elect_mean_epoch_all] * len(psd_periodict_elect_all_epoch_all),
        "slope_elect_mean_epoch_all": [slope_elect_mean_epoch_all] * len(psd_periodict_elect_all_epoch_all),
        'psd_periodict_elect_all_epoch_mean': [psd_periodict_elect_all_epoch_mean] * len(psd_periodict_elect_all_epoch_all),
        'psd_aperiodic_elect_all_epoch_mean': [psd_aperiodic_elect_all_epoch_mean] * len(psd_periodict_elect_all_epoch_all),
        "intercept_y_elect_all_epoch_mean": [intercept_y_elect_all_epoch_mean] * len(psd_periodict_elect_all_epoch_all),
        "slope_elect_all_epoch_mean": [slope_elect_all_epoch_mean] * len(psd_periodict_elect_all_epoch_all)


        })
    
    return table_PLE



In [ ]:
# ## compute power law exponent for each epoch
# def ple_exponent(psds_all, isplot=False):
#     ##prueba con epochs, ahora has de adaptarlo a la función de power_spectrum_nans 
#     # first you need to obtain the frequencies, 
#     freqs= psds_all[0].freqs
#     #for np.polyfit we need to convert the frequencies to array
#     psds_array = [psds.get_data() for psds in psds_all]
#     psds_avg = [np.mean(epoch_psd, axis=0) for epoch_psd in psds_array]
#     psds_grand_avg = np.mean(psds_avg, axis=0)

#     coeffs= np.polyfit(np.log10(freqs), np.log10(psds_grand_avg), deg=1)
#     PLE= -coeffs[0]
#     if isplot==True:
#         #plot of the power spectrum
#         plt.loglog(freqs, psds_grand_avg,  label='PSD Mean')
#         #plot of the coeffs calculated
#         #its 10**y, where y is the linear regression, ax+b, a is slope, x is log10(freqs) and b is the intercept
#         y=coeffs[1] + coeffs[0]*np.log10(freqs)
#         plt.loglog(freqs, 10**y, 'r--', label=f'Fit PLE = {PLE:.2f}')
#         plt.xlabel('Frequency (Hz)')
#         plt.ylabel('Power Spectral Density (dB/Hz)')
#         plt.title('Power Law Exponent Fit using MNE-Python')
#         plt.legend()
#         plt.grid(True)
#         plt.show()

#         plt.show()



#     return coeffs, PLE, psds_grand_avg

In [13]:
table_PLE=compute_ple(epochs, f_range=(0.1, 40),isplot=False)

C:\Users\UCM\AppData\Local\Temp\ipykernel_876\161285217.py:4: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data_epochs = epochs.copy().pick("eeg").get_data()


In [14]:
table_PLE

,Subject,Combination,Epoch,freqs,psd_periodict_elect_all_epoch_all,psd_aperiodic_elect_all_epoch_all,intercept_y_elect_all_epoch_all,slope_elect_all_epoch_all,psd_periodict_elect_mean_epoch_all,psd_aperiodic_elect_mean_epoch_all,intercept_y_elect_mean_epoch_all,slope_elect_mean_epoch_all,psd_periodict_elect_all_epoch_mean,psd_aperiodic_elect_all_epoch_mean,intercept_y_elect_all_epoch_mean,slope_elect_all_epoch_mean
0,S01,S1 - S5,1,"[0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2...","[[1.927724613603628e-12, -1.0965581021399371e-...","[[1.3204044804088218e-11, 1.1949600355614033e-...","[-10.75746992875331, -10.804972335188333, -10....","[-1.1207916113832455, -1.0111336201218473, -1....","[[2.744226524037e-11, -1.4435176230407742e-11,...","[[4.648773904312255e-11, 2.638261006372664e-11...","[-10.59211674996334, -10.486326475275693, -10....","[-1.2887131454560838, -1.28793604236042, -1.58...","[[6.5685896671726554e-12, -1.3233895268768155e...","[[4.854450346639114e-11, 3.6247863177549386e-1...","[-10.541209032893564, -10.514987615725952, -10...","[-1.3599930818592063, -1.3283635544566503, -1...."
1,S01,S1 - S5,2,"[0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2...","[[-2.5649526828539668e-11, -3.1880786049982445...","[[5.017445667615411e-11, 6.876242625671975e-11...","[-10.652719094893449, -10.601326883985886, -10...","[-1.0944337797641246, -1.0625015809016967, -1....","[[2.744226524037e-11, -1.4435176230407742e-11,...","[[4.648773904312255e-11, 2.638261006372664e-11...","[-10.59211674996334, -10.486326475275693, -10....","[-1.2887131454560838, -1.28793604236042, -1.58...","[[6.5685896671726554e-12, -1.3233895268768155e...","[[4.854450346639114e-11, 3.6247863177549386e-1...","[-10.541209032893564, -10.514987615725952, -10...","[-1.3599930818592063, -1.3283635544566503, -1...."
2,S01,S1 - S5,3,"[0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2...","[[-1.3539674096762394e-11, -1.3784718664908355...","[[3.420597650367042e-11, 2.052132149236032e-11...","[-10.694931943199022, -10.71777879769536, -10....","[-1.2501587023070222, -1.174468703276863, -1.2...","[[2.744226524037e-11, -1.4435176230407742e-11,...","[[4.648773904312255e-11, 2.638261006372664e-11...","[-10.59211674996334, -10.486326475275693, -10....","[-1.2887131454560838, -1.28793604236042, -1.58...","[[6.5685896671726554e-12, -1.3233895268768155e...","[[4.854450346639114e-11, 3.6247863177549386e-1...","[-10.541209032893564, -10.514987615725952, -10...","[-1.3599930818592063, -1.3283635544566503, -1...."
3,S01,S1 - S5,4,"[0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2...","[[6.777929147912799e-12, 2.966802454483212e-12...","[[1.1685445190140876e-11, 1.4415657756877727e-...","[-10.718424875464189, -10.719075129550593, -10...","[-1.1928932741592781, -1.1310416535002634, -1....","[[2.744226524037e-11, -1.4435176230407742e-11,...","[[4.648773904312255e-11, 2.638261006372664e-11...","[-10.59211674996334, -10.486326475275693, -10....","[-1.2887131454560838, -1.28793604236042, -1.58...","[[6.5685896671726554e-12, -1.3233895268768155e...","[[4.854450346639114e-11, 3.6247863177549386e-1...","[-10.541209032893564, -10.514987615725952, -10...","[-1.3599930818592063, -1.3283635544566503, -1...."
4,S01,S1 - S5,5,"[0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2...","[[3.395038057743692e-11, -3.623883056952059e-1...","[[7.144627860079275e-11, 6.211951693659292e-11...","[-10.53849857241408, -10.506430981759802, -10....","[-1.4381662124270396, -1.4279108171054005, -1....","[[2.744226524037e-11, -1.4435176230407742e-11,...","[[4.648773904312255e-11, 2.638261006372664e-11...","[-10.59211674996334, -10.486326475275693, -10....","[-1.2887131454560838, -1.28793604236042, -1.58...","[[6.5685896671726554e-12, -1.3233895268768155e...","[[4.854450346639114e-11, 3.6247863177549386e-1...","[-10.541209032893564, -10.514987615725952, -10...","[-1.3599930818592063, -1.3283635544566503, -1...."
5,S01,S1 - S5,6,"[0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2...","[[7.134800349034362e-11, -2.710970719738233e-1...","